In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Git

In [2]:
cd /content/drive/MyDrive/Colab Notebooks/AI_SPARK/AI_SPARK_Challenge_2023

/content/drive/MyDrive/Colab Notebooks/AI_SPARK/AI_SPARK_Challenge_2023


브랜치
- master
- ksw_main

In [3]:
!git branch ksw_main

In [10]:
!git checkout master

Switched to branch 'master'
Your branch is up to date with 'origin/master'.


In [7]:
!git remote update


Fetching origin
Already on 'ksw_main'


In [298]:
!git checkout ksw_main

Switched to branch 'ksw_main'


In [ ]:
!git add -all

In [9]:
!git branch

* ksw_main
  master


# Train

## Import

In [284]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import *

from keras.layers import Input, Dense, BatchNormalization, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.backend import clear_session

## Data Load

In [65]:
df_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI_SPARK/AI_SPARK_Challenge_2023/Data/train_data.csv')

In [181]:
df_data = df_data.drop(axis=1, columns=['out_pressure'])

In [203]:
df_data['type']

array([0, 1, 2, 3, 4, 5, 6, 7])

## AutoEncoder

In [285]:
clear_session()

# configure
encoding_dim = 64
input_data = Input(shape=(7,))

# layers
encoded = Dense(encoding_dim, activation='relu')(input_data)
decoded = Dense(7, activation='linear')(encoded)

# Models
autoencoder = Model(input_data, decoded) # autoencoder

encoder = Model(input_data, encoded) # encoder

encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input)) # decoder

In [286]:
autoencoder.compile(optimizer='adam', loss='mse')

In [287]:
es = EarlyStopping(monitor = 'val_loss',patience=5,restore_best_weights=True, verbose=1)

In [295]:
scaler = StandardScaler()
train_df = scaler.fit_transform(df_data)

In [296]:
autoencoder.fit(train_df, train_df, epochs = 100, callbacks = [es], validation_split = 0.3)

Epoch 1/100
54/54 [==============================] - 0s 4ms/step - loss: 4.5533e-04 - val_loss: 0.0144
Epoch 2/100
54/54 [==============================] - 0s 3ms/step - loss: 4.0496e-04 - val_loss: 0.0142
Epoch 3/100
54/54 [==============================] - 0s 3ms/step - loss: 3.8939e-04 - val_loss: 0.0143
Epoch 4/100
54/54 [==============================] - 0s 3ms/step - loss: 3.5018e-04 - val_loss: 0.0140
Epoch 5/100
54/54 [==============================] - 0s 3ms/step - loss: 3.2810e-04 - val_loss: 0.0141
Epoch 6/100
54/54 [==============================] - 0s 3ms/step - loss: 3.0946e-04 - val_loss: 0.0139
Epoch 7/100
54/54 [==============================] - 0s 3ms/step - loss: 2.8791e-04 - val_loss: 0.0140
Epoch 8/100
54/54 [==============================] - 0s 3ms/step - loss: 2.7380e-04 - val_loss: 0.0140
Epoch 9/100
54/54 [==============================] - 0s 4ms/step - loss: 2.6485e-04 - val_loss: 0.0139
Epoch 10/100
54/54 [==============================] - 0s 3ms/step - loss:

## KFold

In [150]:
kf = KFold(random_state=2023,shuffle=True)

In [161]:
for i, (train_index, test_index) in enumerate(kf.split(df_data)):
  print('='*10)
  print(i)
  print('='*10)
  x_train = df_data.loc[train_index]
  x_test = df_data.loc[test_index]

  scaler = MinMaxScaler()
  x_train = scaler.fit_transform(x_train)
  x_test = scaler.transform(x_test)

  

0
Epoch 1/100
44/44 [==============================] - 3s 9ms/step - loss: 0.1595 - val_loss: 0.2145
Epoch 2/100
44/44 [==============================] - 0s 7ms/step - loss: 0.1369 - val_loss: 0.1875
Epoch 3/100
44/44 [==============================] - 0s 7ms/step - loss: 0.1183 - val_loss: 0.1649
Epoch 4/100
44/44 [==============================] - 0s 7ms/step - loss: 0.1031 - val_loss: 0.1459
Epoch 5/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0909 - val_loss: 0.1306
Epoch 6/100
44/44 [==============================] - 0s 6ms/step - loss: 0.0810 - val_loss: 0.1181
Epoch 7/100
44/44 [==============================] - 0s 6ms/step - loss: 0.0732 - val_loss: 0.1080
Epoch 8/100
44/44 [==============================] - 0s 7ms/step - loss: 0.0669 - val_loss: 0.1001
Epoch 9/100
44/44 [==============================] - 0s 6ms/step - loss: 0.0619 - val_loss: 0.0937
Epoch 10/100
44/44 [==============================] - 0s 6ms/step - loss: 0.0581 - val_loss: 0.0888
Epoch 1